<a href="https://colab.research.google.com/github/Nikkuchan/BleepBot/blob/main/UnnathiBased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from keras.models import Model
from sklearn.metrics import classification_report
from keras import layers, models, optimizers
import sklearn.metrics as metrics
from sklearn.feature_extraction import text 
import pickle


In [135]:
from sklearn.metrics import RocCurveDisplay
from sklearn.datasets import load_wine

In [2]:
#just for word wrap purposes :)
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [3]:
speech_data = pd.read_csv('/content/profanitylabel5CAT.csv')

# split the dataset into training and testing datasets 
x_train, x_test, y_train, y_test = train_test_split(speech_data['Transcript'], speech_data['Category'], test_size=0.3, random_state=1)


# create a count vectorizer object 
my_additional_stop_words = ["akin","ako","mo","ka","tayo","ikaw","alin","amin",
                 "ay","bababa","dapat","ginawa","ginawang","bawat","ano",
                 "dahil","bilang","tama","na","nga","using","si","pati","ang",
                 "hala","pre","nag","edi","punta","pinupuntahan","lang","sa",
                 "mali","-","q","mong","date","busog","kayo","kayong","buka",
                 "ibubuka","nais","try","yung","katropa","gaya","ng","pic","bang",
                 "wala","pakita","computer","ambag","dito","nabawasan","bawas",
                 "footballer", "napaka","sayo","balota","bida","pietro","jett",
                 "mag-text","para","may","hindi","ni","ba","dalawa","salba","tutok",
                 "shadow", "install","ballaran","ito","mga","nalang","noh","baka",
                 "samin","mag","nasa","pa","masa","ata","po","ita","ga","kasi","eh",
                 "ol","nang","kang","naman","ko","tae","ata","atang","sana","kung","di",
                 "ah","much","pag","ni","pagiging","ha","mo","kasi"]

stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)
cv = CountVectorizer(stop_words='english')
cv.fit(speech_data['Transcript'])

# transform the training and testing data using count vectorizer object
x_train_count =  cv.transform(x_train)
x_test_count =  cv.transform(x_test)

tfidf_tr = TfidfTransformer()
x_train_count_tfidf = tfidf_tr.fit_transform(x_train_count)
x_test_count_tfidf = tfidf_tr.transform(x_test_count)

# ngram level tf-idf 
tv_ngram = TfidfVectorizer(ngram_range=(1,2))
tv_ngram.fit(speech_data['Transcript'])
x_train_tfidf_ngram =  tv_ngram.transform(x_train)
x_test_tfidf_ngram =  tv_ngram.transform(x_test)

In [ ]:
print(x_train_count_tfidf)

In [ ]:
print(cv.vocabulary_)

In [4]:
# encode document
vector = cv.transform(speech_data['Transcript'])
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

(500, 477)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [5]:
speech_data['Category'].value_counts()

putangina    100
bobo         100
tanga        100
gago         100
fuck         100
Name: Category, dtype: int64

In [6]:
def classifierModel(classifier, Xtrain, Ytrain, Xtest):
    classifier.fit(Xtrain, Ytrain)
    predictions = classifier.predict(Xtest)
    print("\nAccuracy : ", accuracy_score(predictions, y_test))
    
    target_names = ['bobo', 'tanga','gago','fuck','putangina']
    print(classification_report(y_test, predictions, target_names=target_names))

In [7]:
print("\nNaive Bayes with Count Vectors: ")
classifierModel(MultinomialNB(), x_train_count, y_train, x_test_count)

# print("\nNaive Bayes with Count Vectors + TF-IDF: ")
# classifierModel(MultinomialNB(), x_train_count_tfidf, y_train, x_test_count_tfidf)

# print("\nNaive Bayes with N-Gram Vectors: ")
# classifierModel(MultinomialNB(), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram)


Naive Bayes with Count Vectors: 

Accuracy :  0.9466666666666667
              precision    recall  f1-score   support

        bobo       0.96      0.89      0.93        28
       tanga       0.97      0.97      0.97        31
        gago       0.97      1.00      0.98        30
        fuck       0.94      0.91      0.93        34
   putangina       0.90      0.96      0.93        27

    accuracy                           0.95       150
   macro avg       0.95      0.95      0.95       150
weighted avg       0.95      0.95      0.95       150



In [ ]:
# pickle.dump(MultinomialNB(), open("multiNB.pkl"))

In [ ]:
# print("\nLinearSVM with Count Vectors: ")
# classifierModel(SVC(kernel='linear'), x_train_count, y_train, x_test_count)

# print("\nLinearSVM with Count Vectors + TF-IDF: ")
# classifierModel(SVC(kernel='linear'), x_train_count_tfidf, y_train, x_test_count_tfidf)

# print("\nLinearSVM with N-Gram Vectors: ")
# classifierModel(SVC(kernel='linear'), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram)

In [ ]:
# print("\nRandom Forrest Classifier with Count Vectors: ")
# classifierModel(RandomForestClassifier(random_state=40,n_estimators=1024), x_train_count, y_train, x_test_count)

# print("\nRandom Forrest Classifier with Count Vectors + TF-IDF: ")
# classifierModel(RandomForestClassifier(random_state=40,n_estimators=1024), x_train_count_tfidf, y_train, x_test_count_tfidf)

# print("\nRandom Forrest Classifier with N-Gram Vectors: ")
# classifierModel(RandomForestClassifier(random_state=40,n_estimators=1024), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram)